<a href="https://colab.research.google.com/github/subod4/NLP/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
text = "Pollution is a pressing issue affecting our planet today, encompassing various forms of contaminants that degrade the environment and pose risks to human health. From air and water pollution to soil and noise pollution, its effects are far-reaching and demand immediate attention. Major contributors to pollution include industrial activities, vehicular emissions, agricultural practices, and improper waste disposal, which release harmful substances like carbon dioxide, sulfur dioxide, nitrogen oxides, and pesticides into the environment, contaminating air, water, and soil. Additionally, deforestation and urbanization exacerbate the problem, disrupting ecosystems and natural balance. Pollution has detrimental effects on both the environment and human health, leading to global warming, acid rain, depletion of the ozone layer, and loss of biodiversity. Polluted air and water cause respiratory diseases, cancer, and other health complications, while contaminated soil affects agricultural productivity, further jeopardizing food security. In conclusion, pollution poses a significant threat to the well-being of our planet and its inhabitants. Addressing this issue requires collective efforts from governments, industries, and individuals. Implementing sustainable practices, investing in clean technologies, and raising awareness are crucial steps towards mitigating pollution and preserving the environment for future generations."

In [2]:
len(text)

1440

In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [4]:
nlp = spacy.load('en_core_web_sm')

In [6]:
doc = nlp(text)

In [9]:
tokens = [token.text.lower() for token in doc
          if not token.is_stop and
          not token.is_punct and token.text != '/n']


In [10]:
tokens

['pollution',
 'pressing',
 'issue',
 'affecting',
 'planet',
 'today',
 'encompassing',
 'forms',
 'contaminants',
 'degrade',
 'environment',
 'pose',
 'risks',
 'human',
 'health',
 'air',
 'water',
 'pollution',
 'soil',
 'noise',
 'pollution',
 'effects',
 'far',
 'reaching',
 'demand',
 'immediate',
 'attention',
 'major',
 'contributors',
 'pollution',
 'include',
 'industrial',
 'activities',
 'vehicular',
 'emissions',
 'agricultural',
 'practices',
 'improper',
 'waste',
 'disposal',
 'release',
 'harmful',
 'substances',
 'like',
 'carbon',
 'dioxide',
 'sulfur',
 'dioxide',
 'nitrogen',
 'oxides',
 'pesticides',
 'environment',
 'contaminating',
 'air',
 'water',
 'soil',
 'additionally',
 'deforestation',
 'urbanization',
 'exacerbate',
 'problem',
 'disrupting',
 'ecosystems',
 'natural',
 'balance',
 'pollution',
 'detrimental',
 'effects',
 'environment',
 'human',
 'health',
 'leading',
 'global',
 'warming',
 'acid',
 'rain',
 'depletion',
 'ozone',
 'layer',
 'loss',

In [11]:
tokens1 = []
stopwords = list(STOP_WORDS)
allowed_pos = ['ADJ', 'PROPN', 'VERB', 'NOUN']
for token in doc:
    if token.text not in stopwords and token.text not in punctuation:
        if token.pos_ in allowed_pos:
            tokens1.append(token.text)

In [12]:
tokens1

['Pollution',
 'pressing',
 'issue',
 'affecting',
 'planet',
 'today',
 'encompassing',
 'forms',
 'contaminants',
 'degrade',
 'environment',
 'pose',
 'risks',
 'human',
 'health',
 'air',
 'water',
 'pollution',
 'soil',
 'noise',
 'pollution',
 'effects',
 'reaching',
 'demand',
 'immediate',
 'attention',
 'Major',
 'contributors',
 'pollution',
 'include',
 'industrial',
 'activities',
 'vehicular',
 'emissions',
 'agricultural',
 'practices',
 'improper',
 'waste',
 'disposal',
 'release',
 'harmful',
 'substances',
 'carbon',
 'dioxide',
 'sulfur',
 'dioxide',
 'nitrogen',
 'oxides',
 'pesticides',
 'environment',
 'contaminating',
 'air',
 'water',
 'soil',
 'deforestation',
 'urbanization',
 'exacerbate',
 'problem',
 'disrupting',
 'ecosystems',
 'natural',
 'balance',
 'Pollution',
 'detrimental',
 'effects',
 'environment',
 'human',
 'health',
 'leading',
 'global',
 'warming',
 'acid',
 'rain',
 'depletion',
 'ozone',
 'layer',
 'loss',
 'biodiversity',
 'Polluted',
 'a

In [13]:
from collections import Counter


In [15]:
Counter(tokens)

Counter({'pollution': 7,
         'pressing': 1,
         'issue': 2,
         'affecting': 1,
         'planet': 2,
         'today': 1,
         'encompassing': 1,
         'forms': 1,
         'contaminants': 1,
         'degrade': 1,
         'environment': 4,
         'pose': 1,
         'risks': 1,
         'human': 2,
         'health': 3,
         'air': 3,
         'water': 3,
         'soil': 3,
         'noise': 1,
         'effects': 2,
         'far': 1,
         'reaching': 1,
         'demand': 1,
         'immediate': 1,
         'attention': 1,
         'major': 1,
         'contributors': 1,
         'include': 1,
         'industrial': 1,
         'activities': 1,
         'vehicular': 1,
         'emissions': 1,
         'agricultural': 2,
         'practices': 2,
         'improper': 1,
         'waste': 1,
         'disposal': 1,
         'release': 1,
         'harmful': 1,
         'substances': 1,
         'like': 1,
         'carbon': 1,
         'dioxide': 2,

In [16]:
word_freq = Counter(tokens)

In [17]:
word_freq

Counter({'pollution': 7,
         'pressing': 1,
         'issue': 2,
         'affecting': 1,
         'planet': 2,
         'today': 1,
         'encompassing': 1,
         'forms': 1,
         'contaminants': 1,
         'degrade': 1,
         'environment': 4,
         'pose': 1,
         'risks': 1,
         'human': 2,
         'health': 3,
         'air': 3,
         'water': 3,
         'soil': 3,
         'noise': 1,
         'effects': 2,
         'far': 1,
         'reaching': 1,
         'demand': 1,
         'immediate': 1,
         'attention': 1,
         'major': 1,
         'contributors': 1,
         'include': 1,
         'industrial': 1,
         'activities': 1,
         'vehicular': 1,
         'emissions': 1,
         'agricultural': 2,
         'practices': 2,
         'improper': 1,
         'waste': 1,
         'disposal': 1,
         'release': 1,
         'harmful': 1,
         'substances': 1,
         'like': 1,
         'carbon': 1,
         'dioxide': 2,

In [18]:
max_freq = max(word_freq.values())

In [19]:
max_freq

7

In [20]:
for word in word_freq.keys():
    word_freq[word] = word_freq[word]/max_freq

In [21]:
word_freq

Counter({'pollution': 1.0,
         'pressing': 0.14285714285714285,
         'issue': 0.2857142857142857,
         'affecting': 0.14285714285714285,
         'planet': 0.2857142857142857,
         'today': 0.14285714285714285,
         'encompassing': 0.14285714285714285,
         'forms': 0.14285714285714285,
         'contaminants': 0.14285714285714285,
         'degrade': 0.14285714285714285,
         'environment': 0.5714285714285714,
         'pose': 0.14285714285714285,
         'risks': 0.14285714285714285,
         'human': 0.2857142857142857,
         'health': 0.42857142857142855,
         'air': 0.42857142857142855,
         'water': 0.42857142857142855,
         'soil': 0.42857142857142855,
         'noise': 0.14285714285714285,
         'effects': 0.2857142857142857,
         'far': 0.14285714285714285,
         'reaching': 0.14285714285714285,
         'demand': 0.14285714285714285,
         'immediate': 0.14285714285714285,
         'attention': 0.14285714285714285,
   

In [22]:
sent_token = [sent.text for sent in doc.sents]

In [23]:
sent_token

['Pollution is a pressing issue affecting our planet today, encompassing various forms of contaminants that degrade the environment and pose risks to human health.',
 'From air and water pollution to soil and noise pollution, its effects are far-reaching and demand immediate attention.',
 'Major contributors to pollution include industrial activities, vehicular emissions, agricultural practices, and improper waste disposal, which release harmful substances like carbon dioxide, sulfur dioxide, nitrogen oxides, and pesticides into the environment, contaminating air, water, and soil.',
 'Additionally, deforestation and urbanization exacerbate the problem, disrupting ecosystems and natural balance.',
 'Pollution has detrimental effects on both the environment and human health, leading to global warming, acid rain, depletion of the ozone layer, and loss of biodiversity.',
 'Polluted air and water cause respiratory diseases, cancer, and other health complications, while contaminated soil aff

In [26]:
sent_score = {}
for sent in sent_token:
    for word in sent.split():
      if word.lower() in word_freq.keys():
        if sent not in sent_score.keys():
          sent_score[sent] = word_freq[word]
        else:
          sent_score[sent] += word_freq[word]
      print(word)

Pollution
is
a
pressing
issue
affecting
our
planet
today,
encompassing
various
forms
of
contaminants
that
degrade
the
environment
and
pose
risks
to
human
health.
From
air
and
water
pollution
to
soil
and
noise
pollution,
its
effects
are
far-reaching
and
demand
immediate
attention.
Major
contributors
to
pollution
include
industrial
activities,
vehicular
emissions,
agricultural
practices,
and
improper
waste
disposal,
which
release
harmful
substances
like
carbon
dioxide,
sulfur
dioxide,
nitrogen
oxides,
and
pesticides
into
the
environment,
contaminating
air,
water,
and
soil.
Additionally,
deforestation
and
urbanization
exacerbate
the
problem,
disrupting
ecosystems
and
natural
balance.
Pollution
has
detrimental
effects
on
both
the
environment
and
human
health,
leading
to
global
warming,
acid
rain,
depletion
of
the
ozone
layer,
and
loss
of
biodiversity.
Polluted
air
and
water
cause
respiratory
diseases,
cancer,
and
other
health
complications,
while
contaminated
soil
affects
agricultural
prod

In [27]:
sent_score

{'Pollution is a pressing issue affecting our planet today, encompassing various forms of contaminants that degrade the environment and pose risks to human health.': 2.571428571428571,
 'From air and water pollution to soil and noise pollution, its effects are far-reaching and demand immediate attention.': 2.9999999999999996,
 'Major contributors to pollution include industrial activities, vehicular emissions, agricultural practices, and improper waste disposal, which release harmful substances like carbon dioxide, sulfur dioxide, nitrogen oxides, and pesticides into the environment, contaminating air, water, and soil.': 3.4285714285714275,
 'Additionally, deforestation and urbanization exacerbate the problem, disrupting ecosystems and natural balance.': 0.857142857142857,
 'Pollution has detrimental effects on both the environment and human health, leading to global warming, acid rain, depletion of the ozone layer, and loss of biodiversity.': 2.1428571428571423,
 'Polluted air and wat

In [28]:
import pandas as pd

In [30]:
pd.DataFrame(list(sent_score.items()), columns=['sentence', 'score'])

,sentence,score
0,Pollution is a pressing issue affecting our pl...,2.571429
1,From air and water pollution to soil and noise...,3.000000
2,Major contributors to pollution include indust...,3.428571
3,"Additionally, deforestation and urbanization e...",0.857143
4,Pollution has detrimental effects on both the ...,2.142857
5,Polluted air and water cause respiratory disea...,2.857143
6,"In conclusion, pollution poses a significant t...",1.714286
7,Addressing this issue requires collective effo...,0.714286
8,"Implementing sustainable practices, investing ...",3.000000


In [31]:
from heapq import nlargest

In [32]:
num_sentences = 3
n = nlargest(num_sentences, sent_score, key=sent_score.get)

['Major contributors to pollution include industrial activities, vehicular emissions, agricultural practices, and improper waste disposal, which release harmful substances like carbon dioxide, sulfur dioxide, nitrogen oxides, and pesticides into the environment, contaminating air, water, and soil.',
 'From air and water pollution to soil and noise pollution, its effects are far-reaching and demand immediate attention.',
 'Implementing sustainable practices, investing in clean technologies, and raising awareness are crucial steps towards mitigating pollution and preserving the environment for future generations.']

In [ ]:
"".join(n)